In [ ]:
"""
Algorithm Description:

This algorithm creates the seven filter criteria as defined in the paper. We use our original long criteria as mentioned in 
the paper and use the opposite criteria as short criteria, e.g. only go short in stocks with negative EPS etc.
Runs daily.

"""
#Imports
import quantopian.algorithm as algo
from quantopian.pipeline import Pipeline
from quantopian.algorithm import order_optimal_portfolio
from quantopian.algorithm import attach_pipeline, pipeline_output
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.data import Fundamentals
from quantopian.pipeline.data import morningstar
from quantopian.pipeline.classifiers.fundamentals import Sector
from quantopian.pipeline.filters import QTradableStocksUS
from quantopian.pipeline.factors import SimpleMovingAverage, SimpleBeta, AnnualizedVolatility
from quantopian.pipeline import CustomFactor
import quantopian.optimize as opt
import pandas as pd
import numpy as np

def initialize(context):

    # Daily rebalancing at market open
    algo.schedule_function(
        rebalance,
        date_rule=algo.date_rules.every_day(), 
        time_rule=algo.time_rules.market_open()
    )

    # Creates the pipeline used for selecting our universe
    algo.attach_pipeline(make_pipeline(), 'pipeline')


def make_pipeline():

    universe = QTradableStocksUS()  #American stockmarket 

    # Filter 1: EPS Filter
    eps = Fundamentals.basic_eps_earnings_reports.latest
    eps_filter = eps > 0
    negative_eps_filter = eps < 0

    #Filter 2: Market cap must be larger than 20th percentile
    market_cap = Fundamentals.market_cap.latest
    market_cap_filter = market_cap.percentile_between(20,100)
    negative_market_cap_filter = market_cap.percentile_between(20,100)

    #Filter 3 and 4: Uses crossover of price and 20 day moving average
    ma20 = SimpleMovingAverage(inputs = [USEquityPricing.close], window_length = 20, mask= universe)
    
    price = USEquityPricing.close.latest #For Momentum we use the price as a crossover

    short_ma_filter = price > ma20
    positive_ma_filter = short_ma_filter 

    negative_short_ma_filter = price < ma20
    negative_ma_filter = negative_short_ma_filter 
    
    
    #Filter 5: 1y Beta against SPY has to be below 1 (defensive stocks only)
    #SPY = symbols('SPY') ; seems like this doesnt work as it returns SPY as a list, we use sid(8554)                               now to connect to SPY 
    beta = SimpleBeta(target=sid(8554), regression_length= 252)
    beta_filter = beta < 1
    negative_beta_filter = beta > 1
    
    #Filter 6: Yearly Volatility has to be below the 85 percentile of all assets in the universe
    vola = AnnualizedVolatility(window_length=252)
    vola_filter = vola.percentile_between(0,85)
    negative_vola_filter = vola.percentile_between(85,100)

    #Filter 7: Z-Score of PE ratio
    #Creating the mean of PE grouped by sector for zscore
    class IndustryMeanPE(CustomFactor):  
        inputs = [Fundamentals.pe_ratio, Fundamentals.morningstar_sector_code]  
        window_length = 252

        def compute(self, today, assets, out, pe, industry_codes):  
            df = pd.DataFrame(index=assets, data={"pe_ratio": pe[-1], "industry_codes": industry_codes[-1]}) 

            out[:] = df.groupby("industry_codes").transform(np.mean).values.flatten()
        
    #Creating the standard deviation of the PE grouped by sector for zscore
    class industry_std(CustomFactor):  
        inputs = [Fundamentals.pe_ratio, Fundamentals.morningstar_sector_code]  
        window_length = 252

        def compute(self, today, assets, out, pe, industry_codes):  
            df = pd.DataFrame(index=assets, data={"pe_ratio": pe[-1], "industry_codes": industry_codes[-1]}) 

            out[:] = df.groupby("industry_codes").transform(np.nanstd).values.flatten()
        
    #Getting PE ratio and calulating zscore
    pe = Fundamentals.pe_ratio.latest
    industry_pe = IndustryMeanPE()
    industry_std_of_pe = industry_std()
    zscore = (pe - industry_pe)/industry_std_of_pe
    
    #We only want below average PE ratios so zscore < 0
    zscore_filter = zscore < 0
    negative_zscore_filter = zscore > 0
    
    
    #All combined
    positive_filters = eps_filter & market_cap_filter & positive_ma_filter & beta_filter & vola_filter & zscore_filter
    negative_filters = negative_eps_filter & negative_market_cap_filter & negative_ma_filter & negative_beta_filter & negative_vola_filter & negative_zscore_filter
    
    industry = Fundamentals.morningstar_sector_code.latest
    industry_101 = ~(industry != 101)
    positive_ma_101 = industry_101 & positive_filters
    negative_ma_101 = industry_101 & negative_filters
    industry_102 = ~(industry != 102)
    positive_ma_102 = industry_102 & positive_filters
    negative_ma_102 = industry_102 & negative_filters
    industry_103 = ~(industry != 103)
    positive_ma_103 = industry_103 & positive_filters
    negative_ma_103 = industry_103 & negative_filters
    industry_104 = ~(industry != 104)
    positive_ma_104 = industry_104 & positive_filters
    negative_ma_104 = industry_104 & negative_filters
    industry_205 = ~(industry != 205)
    positive_ma_205 = industry_205 & positive_filters
    negative_ma_205 = industry_205 & negative_filters
    industry_206 = ~(industry != 206)
    positive_ma_206 = industry_206 & positive_filters
    negative_ma_206 = industry_206 & negative_filters  
    industry_207 = ~(industry != 207)
    positive_ma_207 = industry_207 & positive_filters
    negative_ma_207 = industry_207 & negative_filters
    industry_308 = ~(industry != 308)
    positive_ma_308 = industry_308 & positive_filters
    negative_ma_308 = industry_308 & negative_filters
    industry_309 = ~(industry != 309)
    positive_ma_309 = industry_309 & positive_filters
    negative_ma_309 = industry_309 & negative_filters  
    industry_310 = ~(industry != 310)
    positive_ma_310 = industry_310 & positive_filters
    negative_ma_310 = industry_310 & negative_filters
    industry_311 = ~(industry != 311)
    positive_ma_311 = industry_311 & positive_filters
    negative_ma_311 = industry_311 & negative_filters
    
    return Pipeline(
        columns={
            'positive_ma': positive_filters,
            'positive_ma_101': positive_ma_101,
            'positive_ma_102': positive_ma_102,
            'positive_ma_103': positive_ma_103,
            'positive_ma_104': positive_ma_104,
            'positive_ma_205': positive_ma_205,
            'positive_ma_206': positive_ma_206,
            'positive_ma_207': positive_ma_207,
            'positive_ma_308': positive_ma_308,
            'positive_ma_309': positive_ma_309,
            'positive_ma_310': positive_ma_310,
            'positive_ma_311': positive_ma_311,
            'negative_ma': negative_filters,
            'negative_ma_101': negative_ma_101,
            'negative_ma_102': negative_ma_102,
            'negative_ma_103': negative_ma_103,
            'negative_ma_104': negative_ma_104,
            'negative_ma_205': negative_ma_205,
            'negative_ma_206': negative_ma_206,
            'negative_ma_207': negative_ma_207,
            'negative_ma_308': negative_ma_308,
            'negative_ma_309': negative_ma_309,
            'negative_ma_310': negative_ma_310,
            'negative_ma_311': negative_ma_311,
        },
    screen = universe&pe.notnull()&industry_pe.notnull()) #stocks that do not have a PE are filtered out


def compute_target_weights(context, data):
    
    #Here we want to compute our ordering weights
    
    positive_count_101=len(context.positive_ma_101)
    positive_count_102=len(context.positive_ma_102)
    positive_count_103=len(context.positive_ma_103)
    positive_count_104=len(context.positive_ma_104)
    positive_count_205=len(context.positive_ma_205)
    positive_count_206=len(context.positive_ma_206)
    positive_count_207=len(context.positive_ma_207)
    positive_count_308=len(context.positive_ma_308)
    positive_count_309=len(context.positive_ma_309)
    positive_count_310=len(context.positive_ma_310)
    positive_count_311=len(context.positive_ma_311)
    
    negative_count_101=len(context.negative_ma_101)
    negative_count_102=len(context.negative_ma_102)
    negative_count_103=len(context.negative_ma_103)
    negative_count_104=len(context.negative_ma_104)
    negative_count_205=len(context.negative_ma_205)
    negative_count_206=len(context.negative_ma_206)
    negative_count_207=len(context.negative_ma_207)
    negative_count_308=len(context.negative_ma_308)
    negative_count_309=len(context.negative_ma_309)
    negative_count_310=len(context.negative_ma_310)
    negative_count_311=len(context.negative_ma_311)

#Industry filter
    #101: 'Basic Materials',
    #102: 'Consumer Cyclical',
    #103: 'Financial Services',
    #104: 'Real Estate',
    #205: 'Consumer Defensive',
    #206: 'Healthcare',
    #207: 'Utilities',
    #308: 'Communication Services',
    #309: 'Energy',
    #310: 'Industrials',
    #311: 'Technology' ,
    
    #Defining weights according to strategic asset allocation
    short_exposure = -0.3
    long_exposure = 0.7
    weight_sector = 1/11
    positive_weight_sector = long_exposure * weight_sector
    negative_weight_sector = short_exposure * weight_sector
    max_positive_weight_stock = positive_weight_sector / 5
    max_negative_weight_stock = negative_weight_sector / 5
    positive_overweight = (1/20)*long_exposure
    positive_underweight = -(1/20)*long_exposure
    negative_overweight = -(1/20)*short_exposure
    negative_underweight = (1/20)*short_exposure
    
#101
    #Long weight
    positive_nr_101 = positive_count_101
    if positive_nr_101 == 0:
        positive_weight_101 =0
    else:
        positive_weight_101 = (positive_weight_sector+positive_underweight) / positive_nr_101
        if positive_weight_101 > max_positive_weight_stock: 
            positive_weight_101 = max_positive_weight_stock
        else:
            positive_weight_101 = positive_weight_101
     #short weight       
    negative_nr_101 = negative_count_101
    if negative_nr_101 == 0:
        negative_weight_101 =0
    else:
        negative_weight_101 = (negative_weight_sector+negative_underweight) / negative_nr_101
        if negative_weight_101 < max_negative_weight_stock: 
            negative_weight_101 = max_negative_weight_stock
        else:
            negative_weight_101 = negative_weight_101
#102            
    #Long weight
    positive_nr_102 = positive_count_102
    if positive_nr_102 == 0:
        positive_weight_102 =0
    else:
        positive_weight_102 = (positive_weight_sector+positive_underweight) / positive_nr_102
        if positive_weight_102 > max_positive_weight_stock: 
            positive_weight_102 = max_positive_weight_stock
        else:
            positive_weight_102 = positive_weight_102
     #short weight       
    negative_nr_102 = negative_count_102
    if negative_nr_102 == 0:
        negative_weight_102 =0
    else:
        negative_weight_102 = (negative_weight_sector+negative_underweight) / negative_nr_102
        if negative_weight_102 < max_negative_weight_stock: 
            negative_weight_102 = max_negative_weight_stock
        else:
            negative_weight_102 = negative_weight_102
#103
    #Long weight
    positive_nr_103 = positive_count_103
    if positive_nr_103 == 0:
        positive_weight_103 =0
    else:
        positive_weight_103 = (positive_weight_sector+positive_underweight) / positive_nr_103
        if positive_weight_103 > max_positive_weight_stock: 
            positive_weight_103 = max_positive_weight_stock
        else:
            positive_weight_103 = positive_weight_101
     #short weight       
    negative_nr_103 = negative_count_103
    if negative_nr_103 == 0:
        negative_weight_103 =0
    else:
        negative_weight_103 = (negative_weight_sector+negative_underweight) / negative_nr_103
        if negative_weight_103 < max_negative_weight_stock: 
            negative_weight_103 = max_negative_weight_stock
        else:
            negative_weight_103 = negative_weight_103
#104
    #Long weight
    positive_nr_104 = positive_count_104
    if positive_nr_104 == 0:
        positive_weight_104 =0
    else:
        positive_weight_104 = (positive_weight_sector+positive_overweight) / positive_nr_104
        if positive_weight_104 > max_positive_weight_stock: 
            positive_weight_104 = max_positive_weight_stock
        else:
            positive_weight_104 = positive_weight_104
     #short weight       
    negative_nr_104 = negative_count_104
    if negative_nr_104 == 0:
        negative_weight_104 =0
    else:
        negative_weight_104 = (negative_weight_sector+negative_overweight) / negative_nr_104
        if negative_weight_104 < max_negative_weight_stock: 
            negative_weight_104 = max_negative_weight_stock
        else:
            negative_weight_104 = negative_weight_104
#205
    #Long weight
    positive_nr_205 = positive_count_205
    if positive_nr_205 == 0:
        positive_weight_205 =0
    else:
        positive_weight_205 = (positive_weight_sector+positive_overweight) / positive_nr_205
        if positive_weight_205 > max_positive_weight_stock: 
            positive_weight_205 = max_positive_weight_stock
        else:
            positive_weight_205 = positive_weight_205
     #short weight       
    negative_nr_205 = negative_count_205
    if negative_nr_205 == 0:
        negative_weight_205 =0
    else:
        negative_weight_205 = (negative_weight_sector+negative_overweight) / negative_nr_205
        if negative_weight_205 < max_negative_weight_stock: 
            negative_weight_205 = max_negative_weight_stock
        else:
            negative_weight_205 = negative_weight_205
#206
    #Long weight
    positive_nr_206 = positive_count_206
    if positive_nr_206 == 0:
        positive_weight_206 =0
    else:
        positive_weight_206 = (positive_weight_sector+positive_overweight) / positive_nr_206
        if positive_weight_206 > max_positive_weight_stock: 
            positive_weight_206 = max_positive_weight_stock
        else:
            positive_weight_206 = positive_weight_206
     #short weight       
    negative_nr_206 = negative_count_206
    if negative_nr_206 == 0:
        negative_weight_206 =0
    else:
        negative_weight_206 = (negative_weight_sector+negative_overweight) / negative_nr_206
        if negative_weight_206 < max_negative_weight_stock: 
            negative_weight_206 = max_negative_weight_stock
        else:
            negative_weight_206 = negative_weight_206
#207
    #Long weight
    positive_nr_207 = positive_count_207
    if positive_nr_207 == 0:
        positive_weight_207 =0
    else:
        positive_weight_207 = (positive_weight_sector) / positive_nr_207
        if positive_weight_207 > max_positive_weight_stock: 
            positive_weight_207 = max_positive_weight_stock
        else:
            positive_weight_207 = positive_weight_207
     #short weight       
    negative_nr_207 = negative_count_207
    if negative_nr_207 == 0:
        negative_weight_207 =0
    else:
        negative_weight_207 = (negative_weight_sector) / negative_nr_207
        if negative_weight_207 < max_negative_weight_stock: 
            negative_weight_207 = max_negative_weight_stock
        else:
            negative_weight_207 = negative_weight_207
#308
    #Long weight
    positive_nr_308 = positive_count_308
    if positive_nr_308 == 0:
        positive_weight_308 =0
    else:
        positive_weight_308 = (positive_weight_sector) / positive_nr_308
        if positive_weight_308 > max_positive_weight_stock: 
            positive_weight_308 = max_positive_weight_stock
        else:
            positive_weight_308 = positive_weight_308
     #short weight       
    negative_nr_308 = negative_count_308
    if negative_nr_308 == 0:
        negative_weight_308 =0
    else:
        negative_weight_308 = (negative_weight_sector) / negative_nr_308
        if negative_weight_308 < max_negative_weight_stock: 
            negative_weight_308 = max_negative_weight_stock
        else:
            negative_weight_308 = negative_weight_308
#309
    #Long weight
    positive_nr_309 = positive_count_309
    if positive_nr_309 == 0:
        positive_weight_309 =0
    else:
        positive_weight_309 = (positive_weight_sector+positive_underweight) / positive_nr_309
        if positive_weight_309 > max_positive_weight_stock: 
            positive_weight_309 = max_positive_weight_stock
        else:
            positive_weight_309 = positive_weight_309
     #short weight       
    negative_nr_309 = negative_count_309
    if negative_nr_309 == 0:
        negative_weight_309 =0
    else:
        negative_weight_309 = (negative_weight_sector+negative_underweight) / negative_nr_309
        if negative_weight_309 < max_negative_weight_stock: 
            negative_weight_309 = max_negative_weight_stock
        else:
            negative_weight_309 = negative_weight_309
#310
    #Long weight
    positive_nr_310 = positive_count_310
    if positive_nr_310 == 0:
        positive_weight_310 =0
    else:
        positive_weight_310 = (positive_weight_sector) / positive_nr_310
        if positive_weight_310 > max_positive_weight_stock: 
            positive_weight_310 = max_positive_weight_stock
        else:
            positive_weight_310 = positive_weight_310
     #short weight       
    negative_nr_310 = negative_count_310
    if negative_nr_310 == 0:
        negative_weight_310 =0
    else:
        negative_weight_310 = (negative_weight_sector) / negative_nr_310
        if negative_weight_310 < max_negative_weight_stock: 
            negative_weight_310 = max_negative_weight_stock
        else:
            negative_weight_310 = negative_weight_310
#311
    #Long weight
    positive_nr_311 = positive_count_311
    if positive_nr_311 == 0:
        positive_weight_311 =0
    else:
        positive_weight_311 = (positive_weight_sector+positive_overweight) / positive_nr_311
        if positive_weight_311 > max_positive_weight_stock: 
            positive_weight_311 = max_positive_weight_stock
        else:
            positive_weight_311 = positive_weight_311
     #short weight       
    negative_nr_311 = negative_count_311
    if negative_nr_311 == 0:
        negative_weight_311 =0
    else:
        negative_weight_311 = (negative_weight_sector+negative_overweight) / negative_nr_311
        if negative_weight_311 < max_negative_weight_stock: 
            negative_weight_311 = max_negative_weight_stock
        else:
            negative_weight_311 = negative_weight_311 
    
        # Initialize empty target weights dictionary.
    # This will map securities to their target weight.
    weights = {}
    if context.positive_ma_101:
        positive_weight_101 = positive_weight_101
    elif context.positive_ma_102:
        positive_weight_102 = positive_weight_102
    elif context.positive_ma_103:
        positive_weight_103 = positive_weight_103
    elif context.positive_ma_104:
        positive_weight_104 = positive_weight_104
    elif context.positive_ma_205:
        positive_weight_205 = positive_weight_205
    elif context.positive_ma_206:
        positive_weight_206 = positive_weight_206
    elif context.positive_ma_207:
        positive_weight_207 = positive_weight_207
    elif context.positive_ma_308:
        positive_weight_308 = positive_weight_308
    elif context.positive_ma_309:
        positive_weight_309 = positive_weight_309
    elif context.positive_ma_310:
        positive_weight_310 = positive_weight_310
    elif context.positive_ma_311:
        positive_weight_311 = positive_weight_311
    elif context.negative_ma_101:
        negative_weight_101 = negative_weight_101
    elif context.negative_ma_102:
        negative_weight_102 = negative_weight_102
    elif context.negative_ma_103:
        negative_weight_103 = negative_weight_103
    elif context.negative_ma_104:
        negative_weight_104 = negative_weight_104
    elif context.negative_ma_205:
        negative_weight_205 = negative_weight_205
    elif context.negative_ma_206:
        negative_weight_206 = negative_weight_206
    elif context.negative_ma_207:
        negative_weight_207 = negative_weight_207
    elif context.negative_ma_308:
        negative_weight_308 = negative_weight_308
    elif context.negative_ma_309:
        negative_weight_309 = negative_weight_309
    elif context.negative_ma_310:
        negative_weight_310 = negative_weight_310
    elif context.negative_ma_311:
        negative_weight_311 = negative_weight_311       
    else:
        return weights

    # Exit positions in our portfolio if they are not
    # in our longs or shorts lists.

    for security in context.portfolio.positions:
        if security not in context.positive_ma_101 and security not in context.positive_ma_102 and security not in context.positive_ma_103 and security not in context.positive_ma_104 and security not in context.positive_ma_205 and security not in context.positive_ma_206 and security not in context.positive_ma_207 and security not in context.positive_ma_308 and security not in context.positive_ma_309 and security not in context.positive_ma_310 and security not in context.positive_ma_311 and security not in context.negative_ma_101 and security not in context.negative_ma_102 and security not in context.negative_ma_103 and security not in context.negative_ma_104 and security not in context.negative_ma_205 and security not in context.negative_ma_206 and security not in context.negative_ma_207 and security not in context.negative_ma_308 and security not in context.negative_ma_309 and security not in context.negative_ma_310 and security not in context.negative_ma_311 and data.can_trade(security):
            weights[security] = 0

    for security in context.positive_ma_101:  
        weights[security] = positive_weight_101
    for security in context.positive_ma_102:  
        weights[security] = positive_weight_102
    for security in context.positive_ma_103:  
        weights[security] = positive_weight_103
    for security in context.positive_ma_104:  
        weights[security] = positive_weight_104    
    for security in context.positive_ma_205:  
        weights[security] = positive_weight_205
    for security in context.positive_ma_206:  
        weights[security] = positive_weight_206
    for security in context.positive_ma_207:  
        weights[security] = positive_weight_207
    for security in context.positive_ma_308:  
        weights[security] = positive_weight_308
    for security in context.positive_ma_309:  
        weights[security] = positive_weight_309
    for security in context.positive_ma_310:  
        weights[security] = positive_weight_310
    for security in context.positive_ma_311:  
        weights[security] = positive_weight_311
    for security in context.negative_ma_101:  
        weights[security] = negative_weight_101
    for security in context.negative_ma_102:  
        weights[security] = negative_weight_102
    for security in context.negative_ma_103:  
        weights[security] = negative_weight_103
    for security in context.negative_ma_104:  
        weights[security] = negative_weight_104    
    for security in context.negative_ma_205:  
        weights[security] = negative_weight_205
    for security in context.negative_ma_206:  
        weights[security] = negative_weight_206
    for security in context.negative_ma_207:  
        weights[security] = negative_weight_207
    for security in context.negative_ma_308:  
        weights[security] = negative_weight_308
    for security in context.negative_ma_309:  
        weights[security] = negative_weight_309
    for security in context.negative_ma_310:  
        weights[security] = negative_weight_310
    for security in context.negative_ma_311:  
        weights[security] = negative_weight_311


    return weights


def before_trading_start(context, data):
    """
    Called every day before market open. Adds the stocks that passed our filter criteria from the pipeline to trading list.
    """
    pipe_results = pipeline_output('pipeline')

    context.positive_ma_101 = []
    for sec in pipe_results[pipe_results['positive_ma_101']].index.tolist():
        if data.can_trade(sec):
            context.positive_ma_101.append(sec)
    context.positive_ma_102 = []
    for sec in pipe_results[pipe_results['positive_ma_102']].index.tolist():
        if data.can_trade(sec):
            context.positive_ma_102.append(sec)    
    context.positive_ma_103 = []
    for sec in pipe_results[pipe_results['positive_ma_103']].index.tolist():
        if data.can_trade(sec):
            context.positive_ma_103.append(sec)
    context.positive_ma_104 = []
    for sec in pipe_results[pipe_results['positive_ma_104']].index.tolist():
        if data.can_trade(sec):
            context.positive_ma_104.append(sec)    
    context.positive_ma_205 = []
    for sec in pipe_results[pipe_results['positive_ma_205']].index.tolist():
        if data.can_trade(sec):
            context.positive_ma_205.append(sec)
    context.positive_ma_206 = []
    for sec in pipe_results[pipe_results['positive_ma_206']].index.tolist():
        if data.can_trade(sec):
            context.positive_ma_206.append(sec)    
    context.positive_ma_207 = []
    for sec in pipe_results[pipe_results['positive_ma_207']].index.tolist():
        if data.can_trade(sec):
            context.positive_ma_207.append(sec)
    context.positive_ma_308 = []
    for sec in pipe_results[pipe_results['positive_ma_308']].index.tolist():
        if data.can_trade(sec):
            context.positive_ma_308.append(sec)    
    context.positive_ma_309 = []
    for sec in pipe_results[pipe_results['positive_ma_309']].index.tolist():
        if data.can_trade(sec):
            context.positive_ma_309.append(sec)
    context.positive_ma_310 = []
    for sec in pipe_results[pipe_results['positive_ma_310']].index.tolist():
        if data.can_trade(sec):
            context.positive_ma_310.append(sec)
    context.positive_ma_311 = []
    for sec in pipe_results[pipe_results['positive_ma_311']].index.tolist():
        if data.can_trade(sec):
            context.positive_ma_311.append(sec)
    context.negative_ma_101 = []
    for sec in pipe_results[pipe_results['negative_ma_101']].index.tolist():
        if data.can_trade(sec):
            context.negative_ma_101.append(sec)
    context.negative_ma_102 = []
    for sec in pipe_results[pipe_results['negative_ma_102']].index.tolist():
        if data.can_trade(sec):
            context.negative_ma_102.append(sec)    
    context.negative_ma_103 = []
    for sec in pipe_results[pipe_results['negative_ma_103']].index.tolist():
        if data.can_trade(sec):
            context.negative_ma_103.append(sec)
    context.negative_ma_104 = []
    for sec in pipe_results[pipe_results['negative_ma_104']].index.tolist():
        if data.can_trade(sec):
            context.negative_ma_104.append(sec)    
    context.negative_ma_205 = []
    for sec in pipe_results[pipe_results['negative_ma_205']].index.tolist():
        if data.can_trade(sec):
            context.negative_ma_205.append(sec)
    context.negative_ma_206 = []
    for sec in pipe_results[pipe_results['negative_ma_206']].index.tolist():
        if data.can_trade(sec):
            context.negative_ma_206.append(sec)    
    context.negative_ma_207 = []
    for sec in pipe_results[pipe_results['negative_ma_207']].index.tolist():
        if data.can_trade(sec):
            context.negative_ma_207.append(sec)
    context.negative_ma_308 = []
    for sec in pipe_results[pipe_results['negative_ma_308']].index.tolist():
        if data.can_trade(sec):
            context.negative_ma_308.append(sec)    
    context.negative_ma_309 = []
    for sec in pipe_results[pipe_results['negative_ma_309']].index.tolist():
        if data.can_trade(sec):
            context.negative_ma_309.append(sec)
    context.negative_ma_310 = []
    for sec in pipe_results[pipe_results['negative_ma_310']].index.tolist():
        if data.can_trade(sec):
            context.negative_ma_310.append(sec)
    context.negative_ma_311 = []
    for sec in pipe_results[pipe_results['negative_ma_311']].index.tolist():
        if data.can_trade(sec):
            context.negative_ma_311.append(sec)


def rebalance(context, data):
    """
    Execute orders according to our schedule_function() timing.
    """
   
    target_weights = compute_target_weights(context, data)
    if target_weights:
        order_optimal_portfolio(
            objective=opt.TargetWeights(target_weights),
            constraints=[],
        )        
        
